In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-zc39bn9e
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-zc39bn9e
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=c78d859a35a1f5f4b7d8730c7456e3674b262650b3ed2e3e27cbe3058320f769
  Stored in directory: /tmp/pip-ephem-wheel-cache-b9v4128_/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import re

import gensim
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import tensorflow as tf
from nltk import word_tokenize

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Data/CVs/Labeled Data/CV Parser - Labeled Data.csv')
print(data[0:20])

       cv                                               line  tag
0   1.pdf                                    Trinh Dinh Phuc  B-I
1   1.pdf                                   Curriculum Vitae  I-I
2   1.pdf                                16 Kha Van Can Str.  I-I
3   1.pdf                           Linh Dong, Thu Duc, HCMC  I-I
4   1.pdf                                                  H  I-I
5   1.pdf                                      0121 658 5084  I-I
6   1.pdf                                                  B  I-I
7   1.pdf                               Phuccoi996@gmail.com  I-I
8   1.pdf                                DOB: March 16, 1996  I-I
9   1.pdf                                          Education  B-E
10  1.pdf  2014–2018 Bachelor’s Degree of Information Tec...  I-E
11  1.pdf  Telecommunications University (TCU), Nha Trang...  I-E
12  1.pdf                    - CGPA: 3.35/4 via 204 credits.  I-E
13  1.pdf   Thesis: Pneumonia Diagnosis using Lung’s XRay...  I-E
14  1.pdf 

In [ ]:
print("Shape of array: ", data.shape)
print("Total number of CVs: ", data["cv"].nunique())

Shape of array:  (36461, 3)
Total number of CVs:  450


In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(":|-\s+|,|–\s+|-|\||\/", " ", phrase)
    phrase = re.sub(r"â€|™|“|”|;|ü|\xad|\xa0|\u200b|·|∙|�|●|�|§|•|!|▪|©|\?|\]|\[|\)|\(", "", phrase)
    phrase = re.sub(r"[^\w\s]", " ", phrase)
    
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    
    phrase = re.sub(r"\bhcmc\b", "ho chi minh city ", phrase)
    phrase = re.sub(r"\bhcm\b", "ho chi minh ", phrase)
    phrase = re.sub(r"\bbs\b ", "bachelor of ", phrase)
    phrase = re.sub(r"r’s", "r of", phrase)
    phrase = re.sub(r"\’s", " of", phrase)
    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " of", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\bim\b", "i am ", phrase)
    phrase = re.sub(r"\bi’m\b", "i am ", phrase)
    
    return phrase

In [ ]:
line = []
for i in data['line']:
    raw_clean = decontracted(str(i).lower())
    line.append(raw_clean.strip())

In [ ]:
line

['trinh dinh phuc',
 'curriculum vitae',
 '16 kha van can str',
 'linh dong  thu duc  ho chi minh city',
 'h',
 '0121 658 5084',
 'b',
 'phuccoi996 gmail com',
 'dob  march 16  1996',
 'education',
 '2014 2018 bachelor s degree of information technology department of computer science',
 'telecommunications university tcu  nha trang  khanh hoa  vietnam',
 'cgpa  3 35 4 via 204 credits',
 'thesis  pneumonia diagnosis using lung s xray with depthwise convolution',
 'final grade  91 100 top of the class',
 'internship',
 '03 2018  machine learning engineer intern part time intern cbd robotics  125 dong van',
 'cong str  district 2  ho chi minh city  vietnam',
 'work learn about and work on the technical side with focus on data analytics with using machine',
 'learning  deep learning',
 'mini projects house price prediction  amazon reviews  airline arrivals  boston marathon  author',
 'classification',
 '04 2018  7 2018 data analyst intern full time intern devnet  101b mai xuan thuong  vinh

In [ ]:
wv = KeyedVectors.load('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Build_Vocabulary/word2vec_v8.bin', mmap='r')

In [ ]:
def sent2idx(x):
    xidx = []
    for w in word_tokenize(x):
        if w in wv.key_to_index:
            xidx.append(wv.key_to_index[w])
    return np.array(xidx)

In [ ]:
s2v = []
for i in line:
    s2v.append(sent2idx(i))

In [ ]:
for i, x in enumerate(s2v):
    if len(x) < 64:
        s2v[i] = np.hstack((x, np.zeros(64-len(x))))
X = np.array(s2v)
X

array([[ 1363.,   575.,  1155., ...,     0.,     0.,     0.],
       [  253.,   271.,     0., ...,     0.,     0.,     0.],
       [  501.,  3516.,   234., ...,     0.,     0.,     0.],
       ...,
       [  327.,   268.,     0., ...,     0.,     0.,     0.],
       [ 2994.,    80.,   476., ...,     0.,     0.,     0.],
       [ 2401.,     0., 10038., ...,     0.,     0.,     0.]])

In [ ]:
from keras.models import load_model

from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

model = load_model('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Build_Model_Segment/bilstm_crf_v4.h5', custom_objects={'CRF': CRF, 'crf_loss': crf_loss , 
                                                                                                                       'crf_viterbi_accuracy': crf_viterbi_accuracy})

In [ ]:
tag2idx = {'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
idx2tag = {0: 'I-W', 1: 'nan', 2: 'B-SU', 3: 'I-S', 4: 'B-I', 5: 'I_I', 6: 'B-S', 7: 'I-I', 8: 'I-E', 9: 'I-SU', 10: 'B-E', 11: 'B-Su', 12: 'O', 13: 'B-W'}

n_tags = len(tag2idx)
print(tag2idx, '\nn_tag:' ,n_tags)

{'I-W': 0, 'nan': 1, 'B-SU': 2, 'I-S': 3, 'B-I': 4, 'I_I': 5, 'B-S': 6, 'I-I': 7, 'I-E': 8, 'I-SU': 9, 'B-E': 10, 'B-Su': 11, 'O': 12, 'B-W': 13} 
n_tag: 14


In [ ]:
pre_model = []
for i in X:
    p = model.predict(np.array([i]))
    p = np.argmax(p)
    pre_model.append(idx2tag[p])
    print('\r line: ', len(pre_model), end='')
    #print("{:90}: {:5}".format(w, idx2tag[p]))

 line:  36461

In [ ]:
prediction = pre_model

In [ ]:
data['pre_tag_with_model'] = prediction

In [ ]:
data.to_csv('/content/drive/MyDrive/Nguyen Dinh Phuc Dai/Evaluate_Model/data_evaluate.csv')